In [11]:
import numpy as np
import csv
import random
import math

In [12]:
def read_data(file1_path,file2_path):
    dataset1_reader = csv.reader(open(file1_path, encoding='utf-8'))
    dataset2_reader = csv.reader(open(file2_path, encoding='utf-8'))
    # define a list to store the data
    all_data_set = []

    # read the data into a list(name,sequence,class)
    for row in dataset1_reader:
        all_data_set.append([row[0],row[1],row[2]])
    for row in dataset2_reader:
        all_data_set.append([row[0],row[1],row[2]])    
    # shuffle the data set randomly    
    random.seed(2)
    random.shuffle(all_data_set)
    return all_data_set

In [17]:
def vectorize_data(dataset):
    # get the maxmium length of the seqence
    max_seq_len = 0
    for item in dataset:
        if len(item[1])>max_seq_len:
            max_seq_len = len(item[1])
    #print(max_seq_len)
    #max_seq_len = 169
    # padding with "N" to max_seq_len
    for item in dataset:
        item[1] += "N" *(max_seq_len-len(item[1]))
 
    # tranformation of data set:one_hot encoding
    x_cast = {"A":[[1],[0],[0],[0]],"U":[[0],[1],[0],[0]],\
              "T":[[0],[1],[0],[0]],"G":[[0],[0],[1],[0]],\
              "C":[[0],[0],[0],[1]],"N":[[0],[0],[0],[0]]}
    y_cast = {"TRUE": [1,0],"FALSE":[0,1]} #TRUE:Mirtrons  FALSE:canonical microRN
    
    # define a list to store the vectorized data
    
    x=[]
    y=[]
    for item in dataset:
         data = []
         for char in item[1]:
             data.append(x_cast[char])
         #data= np.asarray(data)
    
         x.append(data)
         y.append(y_cast[item[2]])
    return x,y

In [19]:
from sklearn.model_selection import train_test_split
FILE_PATH = "/home/emon/Downloads/cnnMirtronPred-master/data/miRBase_set.csv"
FILE_PATH_PUTATIVE = "/home/emon/Downloads/cnnMirtronPred-master/data/putative_mirtrons_set.csv"

# read datasets and merge as a numpy array
all_data_array = read_data(FILE_PATH,FILE_PATH_PUTATIVE)

# vectorization of the dataset
x,y = vectorize_data(all_data_array)
x= np.array(x)
y=np.array(y)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=.30, random_state=42)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(786, 164, 4, 1)
(786, 2)
(338, 164, 4, 1)
(338, 2)


In [26]:
import tensorflow as tf

#hyperparameters
LR = 0.0001       #learning rate
TRAINING_ITER = 10000   #iteration times
BATCH_SIZE = 18        #batch size of input

SEQUENCE_LENGTH = 164   #sequence length of input
EMBEDDING_SIZE = 4      #char embedding size(sequence width of input)

CONV_SIZE = 3    #first filter size
CONV_DEEP = 128   #number of first filter(convolution deepth)

STRIDES = [1,1,1,1]  #the strid in each of four dimensions during convolution
KSIZE = [1,164,1,1]    #pooling window size

FC_SIZE = 1024     #nodes of full-connection layer
NUM_CLASSES = 2   # classification number

DROPOUT_KEEP_PROB = 0.4   #keep probability of dropout
dataset_size = len(X_train)
# define placeholder
input_X = tf.placeholder(tf.float32,[None,SEQUENCE_LENGTH,EMBEDDING_SIZE,1])
input_y = tf.placeholder(tf.float32,[None, NUM_CLASSES])
keep_prob = tf.placeholder(tf.float32) 

In [27]:
# function of initialize weights
def get_weights_variable(shape):
    initial = tf.truncated_normal(shape,stddev=0.01)
    weights = tf.Variable(initial,name = "weights")
    return weights

In [28]:
# function of convolution and pooling
def conv_and_pooling(input_tensor,filter_height,filter_width,\
                     depth,conv_deep,layer_name):
    
    with tf.name_scope(layer_name):
        conv_weights = get_weights_variable\
                    ([filter_height,filter_width,depth,conv_deep])
        conv_bias = tf.Variable(tf.constant(0.1,shape=[conv_deep]),name = "bias")   
        conv = tf.nn.conv2d(input_tensor,conv_weights,strides = STRIDES,\
                            padding='SAME')
        #print(np.size(conv))
        conv_relu = tf.nn.relu(tf.nn.bias_add(conv,conv_bias))
        conv_relu_pool = tf.nn.max_pool(conv_relu,ksize=KSIZE,strides=STRIDES,padding='VALID')
        #print(np.size(conv_relu_pool))
        # tensorboard visualization
        tf.summary.histogram("../../data/conv_weights",conv_weights)
        tf.summary.histogram("../../data/conv_bias",conv_bias)
        return conv_relu_pool

In [29]:
def fc_output_inference(input_tensor,fc_size,output_size,keep_prob):
    shape_list = input_tensor.get_shape().as_list()
    nodes = shape_list[1]*shape_list[2]*shape_list[3]
    reshaped = tf.reshape(input_tensor,[-1,nodes])

    # the first fully connected layer
    fc1_weights = get_weights_variable([nodes,fc_size])
    fc1_bias = tf.Variable(tf.constant(0.1,shape=[fc_size]))
    fc1 = tf.nn.relu(tf.matmul(reshaped,fc1_weights) + fc1_bias)
    
    # avoid overfitting, droupout regularization
    fc1 = tf.nn.dropout(fc1,keep_prob)
    #fc1 = tf.nn.dropout(fc1,0.5)
 
    # the second fully connected layer(output layer)
    fc2_weights = get_weights_variable([fc_size,output_size])
    fc2_bias = tf.Variable(tf.constant(0.1,shape=[output_size]))
    output = tf.nn.relu(tf.matmul(fc1,fc2_weights) + fc2_bias)

    return output

In [30]:
# use only one kinds of filter in the CNN structure
def cnn_mono_inference(input_tensor,filter_height,filter_width,\
                  in_channels,out_channels,layer_name,keep_prob):
    # layer of convolution and max-pooling
    conv_pool = conv_and_pooling(input_tensor,filter_height,\
                                  filter_width,1,out_channels,layer_name)
 
    output = fc_output_inference(conv_pool,FC_SIZE,NUM_CLASSES,keep_prob)
    
    return output

# use different sizes of filters in the CNN structure
def cnn_concat_inference(input_tensor,filter_height_list,filter_width,\
                         in_channels,out_channels,layer_name_list,keep_prob):
    conv_pool_list = []
    filter_num = len(filter_height_list)
    for i in range(filter_num):
        conv_pool = conv_and_pooling(input_tensor,filter_height_list[i],filter_width,\
                                     in_channels,out_channels,layer_name_list[i]) 
        conv_pool_list.append(conv_pool)

    # concatenant all the conv_pools tensor
    conv_pool_concat = tf.concat([conv_pool for conv_pool in conv_pool_list],1)

    output = fc_output_inference(conv_pool_concat,FC_SIZE,NUM_CLASSES,keep_prob)

    return output

In [31]:
def model_conv3_output(input_X,EMBEDDING_SIZE,keep_prob):
    return cnn_mono_inference(input_X,3,EMBEDDING_SIZE,\
                                           1,128,"Conv3-128",keep_prob)

def model_conv4_output(input_X,EMBEDDING_SIZE,keep_prob):
    return cnn_mono_inference(input_X,4,EMBEDDING_SIZE,\
                                           1,128,"Conv4-128",keep_prob)

def model_conv5_output(input_X,EMBEDDING_SIZE,keep_prob):
    return cnn_mono_inference(input_X,5,EMBEDDING_SIZE,\
                                           1,128,"Conv5-128",keep_prob)


def model_conv6_output(input_X,EMBEDDING_SIZE,keep_prob):
    return cnn_mono_inference(input_X,6,EMBEDDING_SIZE,\
                                           1,128,"Conv6-128",keep_prob)
def model_conv7_output(input_X,EMBEDDING_SIZE,keep_prob):
    return cnn_mono_inference(input_X,7,EMBEDDING_SIZE,\
                                           1,128,"Conv7-128",keep_prob)

def model_concat_output(input_X,EMBEDDING_SIZE,keep_prob):
    concat_output = cnn_concat_inference\
                         (input_X,[3,4,5,6],EMBEDDING_SIZE,1,32,\
                          ["Conv3-32","Conv4-32","Conv5-32","Conv6-32"],keep_prob)
def model_concat2_output(input_X,EMBEDDING_SIZE,keep_prob):
    concat_output = cnn_concat_inference\
                         (input_X,[3,4,5,6,7],EMBEDDING_SIZE,1,32,\
                          ["Conv3-32","Conv4-32","Conv5-32","Conv6-32","Conv7-32"],keep_prob)
    return concat_output

In [32]:
cnn_mono_inference(input_X,3,EMBEDDING_SIZE,1,128,"Conv3-128",keep_prob)
print ("model Conv3-128 was constructed!")
cnn_concat_inference(input_X,[3,4,5,6],EMBEDDING_SIZE,1,32,\
                         ["Conv3-32","Conv4-32","Conv5-32","Conv6-32"],keep_prob)
print ("model Conv-concat was constructed!")

W0915 15:46:25.964170 139925806647104 deprecation.py:506] From <ipython-input-29-bf2d9408005b>:12: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


model Conv3-128 was constructed!
model Conv-concat was constructed!


In [33]:
model_conv3_output(input_X,EMBEDDING_SIZE,keep_prob)
model_conv4_output(input_X,EMBEDDING_SIZE,keep_prob)
model_conv5_output(input_X,EMBEDDING_SIZE,keep_prob)
model_conv6_output(input_X,EMBEDDING_SIZE,keep_prob)
model_concat_output(input_X,EMBEDDING_SIZE,keep_prob)
model_concat2_output(input_X,EMBEDDING_SIZE,keep_prob)

<tf.Tensor 'Relu_15:0' shape=(?, 2) dtype=float32>

In [34]:
def evaluation_op(predic_output,input_ys):
    #calculate TP，TN，FP，FN on test_data
    predictions = tf.argmax(predic_output, 1)
    actuals = tf.argmax(input_ys, 1)

    ones_like_actuals = tf.ones_like(actuals)
    zeros_like_actuals = tf.zeros_like(actuals)
    ones_like_predictions = tf.ones_like(predictions)
    zeros_like_predictions = tf.zeros_like(predictions)

    tn_op = tf.reduce_sum(\
        tf.cast(\
            tf.logical_and(\
            tf.equal(actuals, ones_like_actuals),\
            tf.equal(predictions, ones_like_predictions)\
            ), \
            "float")\
        )

    tp_op = tf.reduce_sum(\
        tf.cast(\
            tf.logical_and(\
            tf.equal(actuals, zeros_like_actuals),\
            tf.equal(predictions, zeros_like_predictions)\
            ),\
            "float")\
        )

    fn_op = tf.reduce_sum(\
        tf.cast(\
          tf.logical_and(\
            tf.equal(actuals, zeros_like_actuals),\
            tf.equal(predictions, ones_like_predictions)\
          ),\
          "float")\
        )

    fp_op = tf.reduce_sum(\
        tf.cast(\
          tf.logical_and(\
            tf.equal(actuals, ones_like_actuals),\
            tf.equal(predictions, zeros_like_predictions)\
          ),\
          "float")\
        )
    
    return predictions,actuals,tp_op, tn_op,fp_op, fn_op



def print_test_evaluation(tp,tn,fp,fn):  
    tpr = float(tp)/(float(tp) + float(fn))
    recall = tpr
    print("Sensitivity/recall on the test data is :{}".format(tpr)) 

    specifity = float(tn)/(float(tn) + float(fp))
    print("specifity on the test data is :{}".format(specifity)) 

    precision = float(tp)/(float(tp) + float(fp))
    print("precision on the test data is :{}".format(precision))

    f1_score = (2 * (precision * recall)) / (precision + recall)
    print("f1_score on the test data is :{}".format(f1_score))

    fpr = float(fp)/(float(tp) + float(fn))
    print("fpr on the test data is :{}".format(fpr))
    if tp!=0 and fp!=0 and tn!=0 and fn!=0:
        mcc = ((float(tp) * float(tn)) - (float(fp) * float(fn))) /\
                math.sqrt((float(tp) + float(fp)) * (float(tp) + float(fn))*\
                 (float(tn) + float(fp)) * (float(tn) + float(fn)))
        print("mcc on the test data is :{}".format(mcc))
    
    
    accuracy = (float(tp) + float(tn))/(float(tp) + float(fp) + float(fn) + float(tn))
    if accuracy > 0.92:
        print("dddddDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDD")
    print("accuracy on the test data is :{}".format(accuracy))
    return accuracy

In [38]:
LR = 0.0001      #learning rate
TRAINING_ITER = 10000   #iteration times
BATCH_SIZE = 18        #batch size of input
 
SEQUENCE_LENGTH = 164   #sequence length of input
EMBEDDING_SIZE = 4      #char embedding size(sequence width of input)
# CONV_SIZE = 3    #first filter size
 # CONV_DEEP = 128   #number of first filter(convolution deepth)
  
STRIDES = [1,1,1,1]  #the strid in each of four dimensions during convolution
KSIZE = [1,164,1,1]    #pooling window size

FC_SIZE = 1024     #nodes of full-connection layer
NUM_CLASSES = 2   # classification number
 
DROPOUT_KEEP_PROB = 0.40   #keep probability of dropout



FILE_PATH = "/home/emon/Downloads/cnnMirtronPred-master/data/miRBase_set.csv"
FILE_PATH_PUTATIVE = "/home/emon/Downloads/cnnMirtronPred-master/data/putative_mirtrons_set.csv"
all_data_array = read_data(FILE_PATH,FILE_PATH_PUTATIVE)

x,y= vectorize_data(all_data_array)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=.30, random_state=42)

X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)
print(y_train.shape)
print(X_train.shape)
print(y_test.shape)
print(X_test.shape)
print("dataset vectorization finished!")
print("iteration",TRAINING_ITER)
dataset_size = len(X_train)  #number of training dataset

input_X = tf.placeholder(tf.float32,[None,SEQUENCE_LENGTH,EMBEDDING_SIZE,1])
input_y = tf.placeholder(tf.float32,[None, NUM_CLASSES])
keep_prob = tf.placeholder(tf.float32)

def train_model_concat2(X_train=X_train,y_train=y_train,X_test=X_test,y_test=y_test,LR=LR,DROPOUT_KEEP_PROB=DROPOUT_KEEP_PROB,dataset_size=dataset_size):
    concat_output = model_concat2_output(input_X,EMBEDDING_SIZE,keep_prob)
    
    #loss_and_optimization(conv6_output,y_train)
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits = concat_output,labels = input_y)
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    # optimization
    train_op = tf.train.RMSPropOptimizer(LR).minimize(cross_entropy_mean)  
    
    # calculate the accuracy of the model
    correct_prediction = tf.equal(tf.argmax(concat_output,1), tf.argmax(input_y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,"float"))
    
    a = tf.placeholder(tf.float32,[None, NUM_CLASSES])
    b = tf.placeholder(tf.float32,[None, NUM_CLASSES])
    auc = tf.metrics.auc(a, b)
    
    tf.summary.scalar("loss",cross_entropy_mean)
    tf.summary.scalar("accuracy",accuracy)

    # evaluation operation test_data
    prd,actu,tp_op,tn_op,fp_op,fn_op = evaluation_op(concat_output,input_y)
 
    merged = tf.summary.merge_all()
    #saver = tf.train.Saver()
    #itr=[]
    #crs=[]
    # train the model with the training dataset
    with tf.Session() as sess:  # run the session        
        #writer = tf.summary.FileWriter(log_path, sess.graph)
        sess.run(tf.global_variables_initializer())
        sess.run(tf.initialize_local_variables())
        for i in range(TRAINING_ITER): 
            start = (i * BATCH_SIZE)% dataset_size
            end = min(start + BATCH_SIZE,dataset_size)
            batch_xs = X_train[start:end]
            batch_ys = y_train[start:end]
            #sess.run(train_op,feed_dict={input_X:batch_xs,input_y:batch_ys,\
            #                              keep_prob:DROPOUT_KEEP_PROB})
            
            _,rs = sess.run([train_op,cross_entropy_mean],\
                            feed_dict={input_X:batch_xs,input_y:batch_ys,\
                                      keep_prob:DROPOUT_KEEP_PROB})
            #writer.add_summary(rs,i)
            
          #  print loss and accuracy during the training process
            #itr.append(i)
            #crs.append(sess.run(cross_entropy_mean,feed_dict={input_X:batch_xs,input_y:batch_ys,keep_prob:DROPOUT_KEEP_PROB}))
            if(i%1000==0):
                print("The {} iteration:".format(i))
                print("The cross_entropy_mean为：",end='')
                print(sess.run(cross_entropy_mean,\
                               feed_dict={input_X:batch_xs,input_y:batch_ys,\
                                          keep_prob:DROPOUT_KEEP_PROB}))
                #print(sess.run(cross_entropy_mean,\
                #               feed_dict={input_X:batch_xs,input_y:batch_ys}))
              #  print("The accuracy on batch data:",end='')
              #  print(sess.run(accuracy,feed_dict={input_xs:batch_xs,\
              #                  input_ys:batch_ys,keep_prob:1}))
                print("The accuracy on training data:",end='')
                print(sess.run(accuracy,feed_dict={input_X:X_train,\
                               input_y:y_train,keep_prob:1}))
               # print(sess.run(accuracy,feed_dict={input_X:X_train,\
                #               input_y:y_train}))
                print("The accuracy on test data:",end='')
                print(sess.run(accuracy,feed_dict={input_X:X_test,\
                                           input_y:y_test,keep_prob:1}))
                print("==================")
                
            #saver.save(sess,model_path)  
        #print(tf.metrics.auc(y_test,correct_prediction))      
        print("*********training finished********")
        print("performance on the test dataset:")
        pr,act,tp,tn,fp,fn = sess.run([prd,actu,tp_op, tn_op,fp_op, fn_op],\
                                feed_dict={input_X:X_test,\
                                input_y:y_test,keep_prob:1})
        print("tp:{},tn:{},fp:{},fn:{}".format(tp,tn,fp,fn))
        #train_auc = sess.run(auc,feed_dict={a:y_test,b:concat_output})
        acc=print_test_evaluation(tp,tn,fp,fn)
        return pr,act,acc

(786, 2)
(786, 164, 4, 1)
(338, 2)
(338, 164, 4, 1)
dataset vectorization finished!
iteration 10000


In [ ]:
#Grid Search 

from sklearn.model_selection import KFold
LR = np.arange(0.0001, 0.001, 0.0001) #Learning rate range from 0.0001 to 0.001
TRAINING_ITER = 10000   #iteration times
BATCH_SIZE = 18        #batch size of input
 
SEQUENCE_LENGTH = 164   #sequence length of input
EMBEDDING_SIZE = 4      #char embedding size(sequence width of input)
# CONV_SIZE = 3    #first filter size
 # CONV_DEEP = 128   #number of first filter(convolution deepth)
  
STRIDES = [1,1,1,1]  #the strid in each of four dimensions during convolution
KSIZE = [1,164,1,1]    #pooling window size

FC_SIZE = 1024     #nodes of full-connection layer
NUM_CLASSES = 2   # classification number
 
DROPOUT_KEEP_PROB = np.arange(0.40, 0.64, 0.04)   #keep probability of dropout


FILE_PATH = "miRBase_set.csv"
FILE_PATH_PUTATIVE = "putative_mirtrons_set.csv"
all_data_array = read_data(FILE_PATH,FILE_PATH_PUTATIVE)
x,y,vectorized_dataset = vectorize_data(all_data_array)
#X_train, y_train, X_test, y_test = data_partition(vectorized_dataset)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=.30, random_state=42)

X_train = np.array(X_train)
y_train = np.array(y_train)
X_test =np.array(X_test)
y_test = np.array(y_test)
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)
print(y_train.shape)
print(X_train.shape)
print(y_test.shape)
print(X_test.shape)
print("dataset vectorization finished!")
print("iteration",TRAINING_ITER)
dataset_size = len(X_train)  #number of training dataset

input_X = tf.placeholder(tf.float32,[None,SEQUENCE_LENGTH,EMBEDDING_SIZE,1])
input_y = tf.placeholder(tf.float32,[None, NUM_CLASSES])
keep_prob = tf.placeholder(tf.float32)
kfold = KFold(5, True, 1)

# enumerate splits
m=0
a1=a2=0
lrs=[]
drs=[]
acs=[]
for l in LR:
    for d in DROPOUT_KEEP_PROB:
        print("Learning Rate: ",l)
        print("Dropout: ",d,"\n\n")
        lrs.append(d)
        drs.append(l)
        s=0
        for train, test in kfold.split(X_train):
            xtr, xts = X_train[train], X_train[test]
            ytr, yts = y_train[train], y_train[test]
            dataset_size = len(xtr)
            print(xtr.shape)
            print(xts.shape)
            print(ytr.shape)
            print(yts.shape)
            _,_,a=train_model_concat2(xtr,ytr,xts,yts,l,d,dataset_size)
            s=s+float(a)
        ac=s/5
        acs.append(ac)
        print("AAAAAAAAAAA: ",ac)
        print("\n\n")
        if m<ac:
            m=ac
            a1=l
            a2=d
print(a1)
print(a2)

In [ ]:
pr,au,ac=train_model_concat2(X_train,y_train,X_test,y_test,0.0001,0.40,len(X_train))

In [ ]:
#Plotting ROC curve
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

FPR, TPR, _ = roc_curve(ac6, pr6)
AUC = auc(FPR, TPR)
#f1,t1,_= roc_curve(act2, pr2)
#AUC2 = auc(f1, t1)
# calculate roc curve
plt.figure()
plt.plot(FPR,TPR,label='ROC curve (area = %0.3f)' % AUC)
#plt.plot(f1,t1,label='ROC curve concat(area = %0.3f)' % AUC2)
plt.plot([0, 1], [0, 1], 'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.02])
plt.grid(True,'both')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.savefig('image.jpg')
plt.show()
#plt.savefig('testplot.png')
#Image.open('testplot.png').save('testplot.jpg','JPEG')